# Projet maison n° 4

In [831]:
# imports
import pandas as pd
import numpy as np


## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas, pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [832]:
# names us
def df_names_us():
    path = "/Users/valentinadiproietto/Google Drive/Cambio/TelecomParisTech/MDI721KitDataScience/Settimana4/names"
    #all_files is a list, every element of the list is a string, given by the path of a file in the folders names which ends with txt
    all_files = glob.glob(path + "/*.txt")
    #I create an empy list li
    li = []
    #for every file .txt I create a dataframe as required and I append it to the list li
    for filename in all_files:
        dfsmall = pd.read_csv(filename, index_col=None, header = None)
   
        dfsmall.columns= ["name", "gender", "births"]
        dfsmall["year"]= filename[-8:-4]
        dfsmall=dfsmall[["year", "name", "gender", "births"]]
    
        li.append(dfsmall)
    #I concatenate each dataframe into a big dataframe df    
    df = pd.concat(li, axis=0)
    #I sort the dataframe as required
    df=df.sort_values(by=['year', 'gender', 'births','name' ], ascending=[True, True, False, True])
    #I change the index in such a way it goes from 0 to len(df)
    df=df.reset_index(drop= True)
   


    
    return df



## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2019_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [833]:
# names fr
def df_names_fr():
    df = pd.DataFrame()
    df = pd.read_csv("nat2019.csv", sep=";", header =0 , names= ['gender', 'name', 'year', 'births'])
    
    #I rearrange the colums as required
    df=df[["year", "name", "gender", "births"]]
    
    #I realised that pyton read the name "NA" as float, and it puts a Nan at their place! So we do the following 
    df.loc[df['name'].apply(lambda x: isinstance(x, float)), 'name'] = 'Na'
    
    #I change sex from 1,2 to M,F
    mapping = {1: 'M', 2: 'F'}
    df['gender'] = df['gender'].map(mapping)
    
    #I erase the rows which have as names "_PRENOMS_RARES"
    
    df= df[df['name']!="_PRENOMS_RARES"]
    
    #I erase the rows which have as names a string with less than 2 character
    df=df[df['name'].str.len()>=2]
    
    #I erase the rows which have as year a XXXX
    df= df[df['year']!="XXXX"]
    
    #I chaneh the name as a string with only the first letter capital 
    df['name']=df['name'].apply(lambda x: str(x).lower().capitalize())
    
    #I sort, as in the us case
    df=df.sort_values(by=['year', 'gender', 'births','name' ], ascending=[True, True, False, True])
        
    df=df.reset_index(drop= True)

    return df

## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/browseBox.do?node=5385566

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Télécharger le dataset des taux de change : http://webstat.banque-france.fr/fr/downloadFile.do?id=5385698&exportType=csv


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [923]:
# taux de change
def df_taux_change(devises):
    
    #I import the data from line 6
    df = pd.read_csv("Webstat_Export_20201025.csv", sep =";", header = 5)
    #I change the index into the format '%d/%m/%Y'
    df["Source :"]= pd.to_datetime(df["Source :"], format= '%d/%m/%Y')
    #I change the names to the columns 
    first_line= pd.read_csv("Webstat_Export_20201025.csv", sep =";").iloc[0]
    df.columns =first_line.apply(lambda x : x[6:9])
    #I rename the first column as 'Dates'
    df = df.rename(columns={'éri': 'Dates'})
    #I put the colum 'Dates as the index'
    df = df.set_index(df['Dates'])
    
    df=df.drop(columns= 'Dates')
    df.index.name = None
    #I replace all the "," with "."
    df = df.apply(lambda x: x.str.replace(',','.'))
    #I erase all the row for which I the first column ther is a string whcih contains "-"
    df=df[~df["AUD"].str.contains("-")]
    #I erase all the lines for which there is a  NaN
    df=df.dropna()
    #I cahneg all the values into numerical values
    df =df.apply(pd.to_numeric)
    return df[devises]

### Tests

In [939]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 1989401)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 615912)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)


In [940]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [941]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 6.454s

OK
